In [2]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

/home/droca1/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Development

In [46]:
def get_child_directions_advanced(t):
    labels = []
    inorder_traversal = []
    C_Tree.inorder(t,  lambda x: inorder_traversal.append(x))
    labels = [x.label for x in inorder_traversal]
    return labels
tree_string = "(S (CC But) (SBAR (IN while) (S (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)) (VP (VBD did) (RB n't) (VP (VB fall) (ADVP (RB apart)) (NP (NNP Friday)) (SBAR (IN as) (S (NP (DT the) (NNP Dow) (NNP Jones) (NNP Industrial) (NNP Average)) (VP (VBD plunged) (NP (NP (CD 190.58) (NNS points)) (PRN (: --) (NP (NP (JJS most)) (PP (IN of) (NP (PRP it))) (PP (IN in) (NP (DT the) (JJ final) (NN hour)))) (: --)))))))))) (NP (PRP it)) (ADVP (RB barely)) (VP (VBD managed) (S (VP (TO to) (VP (VB stay) (NP (NP (DT this) (NN side)) (PP (IN of) (NP (NN chaos)))))))) (. .))"
constituent_tree = C_Tree.from_string(tree_string)
binary_tree = C_Tree.to_binary_right(constituent_tree)
binary_tree_collapsed = binary_tree.collapse_unary()
binary_tree_collapsed = binary_tree_collapsed.remove_preterminals()
binary_tree_collapsed = C_Tree(C_ROOT_LABEL, binary_tree_collapsed)
Tree.fromstring(str(binary_tree_collapsed)).pretty_print()
lbls = get_child_directions_advanced(binary_tree_collapsed)
print(lbls)

                                                                                                                                    -ROOT-                                                                                                                                
                                                                                                                                      |                                                                                                                                    
                                                                                                                                      S                                                                                                                                   
  ____________________________________________________________________________________________________________________________________|__________                                                     

In [42]:
import re
import copy

directions_dir = {"lL":0,"lR":1,"rL":2,"rR":3}
# def get_child_directions(t):
#     labels = []
#     for node in t.children:
#         if node.is_terminal():
#             label_string = ""

#             if node.is_right_child():
#                 label_string+="l"
#             elif node.is_left_child():
#                 label_string+="r"
            
#             if node.parent.is_right_child():
#                 label_string+="L"
#             elif node.parent.is_left_child():
#                 label_string+="R"
            
#             labels.append(label_string)
#         child_labels = get_child_directions(node)
#         for label in child_labels if child_labels != [] else []:
#             labels.append(label)
    
#     return labels

def get_child_directions_advanced(t):
    labels = []
    inorder_traversal = []
    C_Tree.inorder(t,  lambda x: inorder_traversal.append(x))
    return labels



def encode(constituent_tree):        
    lc_tree = LinearizedTree.empty_tree()

    # Compute the Binary Tree and the arrows
    binary_tree = C_Tree.to_binary_right(constituent_tree)
    
    binary_tree_collapsed = binary_tree.collapse_unary()
    binary_tree_collapsed = binary_tree_collapsed.remove_preterminals()
    binary_tree_collapsed = C_Tree(C_ROOT_LABEL, binary_tree_collapsed)
    child_dirs = get_child_directions_advanced(binary_tree_collapsed)

    # Compute the number of commons
    leaf_paths = binary_tree.path_to_leaves(collapse_unary=True, unary_joiner="+")
    
    for i in range(0, len(leaf_paths)-1):
        path_a = leaf_paths[i]
        path_b = leaf_paths[i+1]
        
        last_common = ""
        for a,b in zip(path_a, path_b):
            if (a!=b):
                # Remove the digits and aditional feats in the last common node
                last_common = re.sub(r'[0-9]+', '', last_common)
                last_common = last_common.split("##")[0]

                # Get word and POS tag
                word = path_a[-1]
                postag = path_a[-2]
                
                # Build the Leaf Unary Chain
                unary_chain = None
                leaf_unary_chain = postag.split("+")
                if len(leaf_unary_chain)>1:
                    unary_list = []
                    for element in leaf_unary_chain[:-1]:
                        unary_list.append(element.split("##")[0])

                    unary_chain ="+".join(unary_list)
                    postag = leaf_unary_chain[len(leaf_unary_chain)-1]
                
                # Clean the POS Tag and extract additional features
                postag_split = postag.split("##")
                feats = [None]

                if len(postag_split) > 1:
                    postag = re.sub(r'[0-9]+', '', postag_split[0])
                    feats = postag_split[1].split("|")
                else:
                    postag = re.sub(r'[0-9]+', '', postag)

                direction = child_dirs[i]
                c_label = C_Label(direction, last_common, unary_chain, C_TETRA_ENCODING, "_", "+")
                
                # Append the data
                lc_tree.add_row(word, postag, feats, c_label)
            
                break            
            last_common = a
        
    # n = max number of features of the tree
    lc_tree.n = max([len(f) for f in lc_tree.additional_feats])
    return lc_tree

tree_string = "(S (CC But) (SBAR (IN while) (S (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)) (VP (VBD did) (RB n't) (VP (VB fall) (ADVP (RB apart)) (NP (NNP Friday)) (SBAR (IN as) (S (NP (DT the) (NNP Dow) (NNP Jones) (NNP Industrial) (NNP Average)) (VP (VBD plunged) (NP (NP (CD 190.58) (NNS points)) (PRN (: --) (NP (NP (JJS most)) (PP (IN of) (NP (PRP it))) (PP (IN in) (NP (DT the) (JJ final) (NN hour)))) (: --)))))))))) (NP (PRP it)) (ADVP (RB barely)) (VP (VBD managed) (S (VP (TO to) (VP (VB stay) (NP (NP (DT this) (NN side)) (PP (IN of) (NP (NN chaos)))))))) (. .))"
gold_tree = C_Tree.from_string(tree_string)

lc = encode(gold_tree)
c = 1
debug = [21,25,37]
for w,p,f,l in lc.iterrows():
    if c in debug:
        print("--------------------")
    print(c,str(l)[0],str(l)[1],w)
    if c in debug:
        print("--------------------")
    c+=1

node But was inserted by S The parent of the tree is True
node while was inserted by SBAR The parent of the tree is True
node the was inserted by NP The parent of the tree is True
node New was inserted by NP* The parent of the tree is True
node York was inserted by NP** The parent of the tree is True
node Stock was inserted by NP*** The parent of the tree is True
node Exchange was inserted by NP*** The parent of the tree is True
node did was inserted by VP The parent of the tree is True
node n't was inserted by VP* The parent of the tree is True
node fall was inserted by VP The parent of the tree is True
node apart was inserted by VP* The parent of the tree is True
node Friday was inserted by VP** The parent of the tree is True
node as was inserted by SBAR The parent of the tree is True
node the was inserted by NP The parent of the tree is True
node Dow was inserted by NP* The parent of the tree is True
node Jones was inserted by NP** The parent of the tree is True
node Industrial was 

In [3]:
import copy

def combine(tree, new_child):
    '''
    Replaces a C_NONE_LABEL inside 'tree'
    with new_child
    '''
    # trees should have only 2 child nodes
    if type(new_child) is str:
        new_child = C_Tree(new_child)
    
    current_level = tree
    
    while(not current_level.has_none_child()):
        current_level = current_level.r_child()

    if current_level.children[0].label == C_NONE_LABEL:
        current_level.children[0] = new_child
    elif current_level.children[1].label == C_NONE_LABEL:
        current_level.children[1] = new_child
    return tree

def build_unary_chain(word, postag, unary_chain, unary_joiner):
    if unary_chain:
        unary_chain = unary_chain.split(unary_joiner)
        unary_chain.reverse()
        pos_tree = C_Tree(postag, C_Tree(word))
        for node in unary_chain:
            temp_tree = C_Tree(node, pos_tree)
            pos_tree = temp_tree
    else:
        pos_tree = C_Tree(postag, C_Tree(word))
    return pos_tree

def decode(lc):
    stack = []
    buffer = copy.deepcopy(lc.words)
    tree = None
    for word, postag, feats, label in lc.iterrows():
        a, t, uc = label.n_commons, label.last_common, label.unary_chain
        a1, a2 = a[0], a[1]
        ########
        # BUILD LEAF-TREE WITH POSTAGS AND UNARY CHAIN
        ########
        if a1 == "r":
            leaf = buffer.pop(0)
            terminal_tree = build_unary_chain(leaf, postag, uc, "+")
            stack.append(terminal_tree)
        
        if a1 == "l":
            leaf = buffer.pop(0)
            terminal_tree = build_unary_chain(leaf, postag, uc, "+")
            stack[-1] = combine(stack[-1], terminal_tree)


        # whenever the buffer is empty, we STOP. We should perfom
        # (2 * |w| - 1) actions
        
        if len(buffer)==0:
            break

        if a2 == "R":
            tree = C_Tree(t, [stack[-1], C_Tree.empty_tree()])
            stack[-1] = tree
            
        if a2 == "L":
            tree = stack.pop()
            tree = C_Tree(t, [tree, C_Tree.empty_tree()])
            stack[-1] = combine(stack[-1], tree)
            
    return stack[0]

tree = decode(lc)
print(tree)
tree = C_Tree.restore_from_binary(tree)

nltk_t = Tree.fromstring(str(tree))
nltk_t.pretty_print()


(S (INTJ (RB No)) (S* (, ,) (S** (NP (PRP it)) (S*** (VP (VBD was) (VP* (RB n't) (NP (NNP Black) (NNP Monday)))) (. .)))))
              S                          
  ____________|________________________   
 |    |   |       VP                   | 
 |    |   |    ___|_________           |  
INTJ  |   NP  |   |         NP         | 
 |    |   |   |   |     ____|____      |  
 RB   ,  PRP VBD  RB  NNP       NNP    . 
 |    |   |   |   |    |         |     |  
 No   ,   it was n't Black     Monday  . 



## Testing from file

In [32]:
tetraencoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False)
tree_string = "(S (CC But) (SBAR (IN while) (S (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)) (VP (VBD did) (RB n't) (VP (VB fall) (ADVP (RB apart)) (NP (NNP Friday)) (SBAR (IN as) (S (NP (DT the) (NNP Dow) (NNP Jones) (NNP Industrial) (NNP Average)) (VP (VBD plunged) (NP (NP (CD 190.58) (NNS points)) (PRN (: --) (NP (NP (JJS most)) (PP (IN of) (NP (PRP it))) (PP (IN in) (NP (DT the) (JJ final) (NN hour)))) (: --)))))))))) (NP (PRP it)) (ADVP (RB barely)) (VP (VBD managed) (S (VP (TO to) (VP (VB stay) (NP (NP (DT this) (NN side)) (PP (IN of) (NP (NN chaos)))))))) (. .))"

gold_tree = C_Tree.from_string(tree_string)
bt = str(C_Tree.to_binary_right(gold_tree))
print(bt)
lin_tree = tetraencoder.encode(gold_tree)

c = 1
debug = [21,25,37]
for w,p,f,l in lc.iterrows():
    if c in debug:
        print("--------------------")
    print(c,str(l)[0],str(l)[1],w)
    if c in debug:
        print("--------------------")
    c+=1


(S (CC But) (S* (SBAR (IN while) (S (NP (DT the) (NP* (NNP New) (NP** (NNP York) (NP*** (NNP Stock) (NNP Exchange))))) (VP (VBD did) (VP* (RB n't) (VP (VB fall) (VP* (ADVP (RB apart)) (VP** (NP (NNP Friday)) (SBAR (IN as) (S (NP (DT the) (NP* (NNP Dow) (NP** (NNP Jones) (NP*** (NNP Industrial) (NNP Average))))) (VP (VBD plunged) (NP (NP (CD 190.58) (NNS points)) (PRN (: --) (PRN* (NP (NP (JJS most)) (NP* (PP (IN of) (NP (PRP it))) (PP (IN in) (NP (DT the) (NP* (JJ final) (NN hour)))))) (: --)))))))))))))) (S** (NP (PRP it)) (S*** (ADVP (RB barely)) (S**** (VP (VBD managed) (S (VP (TO to) (VP (VB stay) (NP (NP (DT this) (NN side)) (PP (IN of) (NP (NN chaos)))))))) (. .))))))
1 r R But
2 r R while
3 r R the
4 r L New
5 r L York
6 r L Stock
7 l L Exchange
8 r L did
9 r L n't
10 r L fall
11 r L apart
12 r L Friday
13 r L as
14 r R the
15 r L Dow
16 r L Jones
17 r L Industrial
18 l L Average
19 r L plunged
20 r R 190.58
--------------------
21 l R points
--------------------
22 r L --
23 r 

In [7]:
dec_tree = tetraencoder.decode(lin_tree)
print(dec_tree)

(S (CC But) -NONE-)
